# Exract Player Profile Information

The First two datasets we used to build our Player Profiles were all csv files from Kaggle.com('Players' and 'player_data').
   We want Player Names, Career Positions, Height in Centimeters, Weight in Pounds, College, City and State of Birth.
   than we weill merge the two dataframes together, then agian with the latitude and Longitude of 50 US states
   

In [1]:
#dependencies
import numpy as np
import pandas as pd
import os
import csv

In [2]:
# We Import NBA PLayer's Career Profile
#    This will tell us where they are from

players = './resources/Players.csv'
career= pd.read_csv(players)

# print('Number of Rows', len(career))
career.count()

Unnamed: 0     3922
Player         3921
height         3921
weight         3921
collage        3573
born           3921
birth_city     3452
birth_state    3439
dtype: int64

In [3]:
#  We Import NBA Player's Season Profiles
#    This will tell us where they played

player_data = './resources/player_data.csv'
plyr_season = pd.read_csv(player_data)

# Renames the Column 'name' to 'Player'
plyr_season = plyr_season.rename(columns={'name': 'Player'})

# print('Number of Rows', len(plyr_season))
plyr_season.count()

Player        4550
year_start    4550
year_end      4550
position      4549
height        4549
weight        4544
birth_date    4519
college       4248
dtype: int64

In [4]:
# Merge the plyr_season & career Dataframes on 'Players'

Player_Profile_Dirty = pd.merge(plyr_season, career, how='left', on=["Player","Player"])

print('Number of Rows', len(Player_Profile_Dirty))
Player_Profile_Dirty.count()

Number of Rows 4550


Player         4550
year_start     4550
year_end       4550
position       4549
height_x       4549
weight_x       4544
birth_date     4519
college        4248
Unnamed: 0     3814
height_y       3814
weight_y       3814
collage        3497
born           3814
birth_city     3367
birth_state    3353
dtype: int64

In [33]:
# Create DataFrames and edit Column Names


new_columns = [
'Player',
'birth_state',
'position']
PP= Player_Profile_Dirty[new_columns]
# Renames Columns
PPD = PP.rename(columns={
    'birth_state':'State',
    'position':'Position(career)'})
print('length of PPD', len(PPD))



# -----------
htwt = [
    'Player',
    'height_y',
    'weight_x']
player_htwt= Player_Profile_Dirty[htwt]
# Renames Columns
P_htwt = player_htwt.rename(columns={
    'height_y':'Height_cm',
    'weight_x':'Weight_lbs'})
print('length of P_htwt', len(P_htwt))



# -----------
college = [
    'Player',
    'college']
player_college= Player_Profile_Dirty[college]
# Renames Columns
P_college = player_college.rename(columns={
   'college':'College'})
print('length of P_college', len(P_college))



# PPD.count()
# P_htwt.count()
# P_college.count()

length of PPD 4550
length of P_htwt 4550
length of P_college 4550


In [34]:
# We Import United States latitude and Longitude

LL= './resources/50_SLat_Lng.csv'
ltlg= pd.read_csv(LL)

# Renames Colums
lat_lng= ltlg.rename(columns={
    'Place Name':'State'})
# lat_lng

In [35]:
#  Merge Player Profile Dataframe  with Lat. & Lng Dataframe

Player_Profile_State = pd.merge(PPD,lat_lng, how='left', on=["State","State"])
# print(len(Player_Profile_State))
Player_Profile_State.count()

Player              4550
State               3353
Position(career)    4549
Latitude            2928
Longitude           2928
dtype: int64

In [36]:
# Remove Players not Born in the USA or info not recorded (1622)
# Remove Duplicate Players by -> name and state

Player_State = Player_Profile_State.dropna(axis = 0, how ='any') 


print("Old data frame length:", len(Player_Profile_State),
      "\nNew data frame length:", len(Player_State),
      "\nNumber of Player Not Born In USA: ",(len(Player_Profile_State)-len(Player_State)))
# Player_Profile_State.count()
Player_State.count()

Old data frame length: 4550 
New data frame length: 2927 
Number of Player Not Born In USA:  1623


Player              2927
State               2927
Position(career)    2927
Latitude            2927
Longitude           2927
dtype: int64

In [37]:
# ADD USA BORN PLAYERS  College Height and Weight
# Remove Duplicate Players by -> name and state
# Remove player with imcomplete Ht & Wt Data


Player_htwt = pd.merge(Player_State, P_htwt, how='left', on=["Player","Player"])

Player_htwt.drop_duplicates(subset =('Player', 'State'),
                            keep = False, inplace = True)
Player_Profile = Player_htwt.dropna(axis = 0, how ='any') 

print("Old data frame length:", len(Player_State),
      "\nNew data frame length:", len(Player_Profile),
      "\nNumber of Players with Incomplete Ht&Wt Data: ",len(Player_State)-len(Player_Profile))

Player_Profile.count()

Old data frame length: 2927 
New data frame length: 2844 
Number of Players with Incomplete Ht&Wt Data:  83


Player              2844
State               2844
Position(career)    2844
Latitude            2844
Longitude           2844
Height_cm           2844
Weight_lbs          2844
dtype: int64

In [38]:
# Time to Add Statistics !!!



# We Import NBA Teams Location

tll = './resources/NBATeamLL.csv'
Team_LL= pd.read_csv(tll)
# Team_LL

In [39]:
# # We Import NBA Player Season Stats

Season_stats = './resources/Seasons_Stats.csv'
stats_season = pd.read_csv(Season_stats)
#  Renames Columns

dirty_stats = stats_season.rename(columns={
    'Tm': 'Team',
    'TRB':'Rebounds',
    'STL':'Steals',
    'BLK':'Blocks',
    'AST':'Assists',
    'PTS':'Points',
    'PER':'Player Rating'})
# dirty_stats

In [40]:
season_stats = pd.merge(dirty_stats, Team_LL, how='left', on=["Team","Team"])
# season_stats.count()

In [41]:
# Select only the Columns that are needed.

new_columns = [
    'Year',
    'Player',
    'Team',
    'Team Name',
    'Team Lat',
    'Team Lng',
    'Rebounds',
    'Steals',
    'Blocks',
    'Assists',
    'Points',
    'Player Rating'
]
stats_col= season_stats[new_columns]
# stats_col

In [42]:
# Select only the 2010-2015 Seaason statistic

STATS = stats_col.loc[(stats_col['Year']>=2010)&(stats_col['Year']<=2015)]
STATS.count()
# STATS.to_csv('check.csv')

Year             3589
Player           3589
Team             3589
Team Name        3589
Team Lat         3589
Team Lng         3589
Rebounds         3589
Steals           3589
Blocks           3589
Assists          3589
Points           3589
Player Rating    3587
dtype: int64

In [25]:
# Merge the Clean PLayer Profile with the Clean Player Stats

NBA = pd.merge(STATS,Player_Profile, how='left', on=["Player","Player"])
NBA = NBA.dropna(axis = 0, how ='any')
NBA.count()

Year             2686
Player           2686
Team             2686
Team Name        2686
Team Lat         2686
Team Lng         2686
Rebounds         2686
Steals           2686
Blocks           2686
Assists          2686
Points           2686
Player Rating    2686
State            2686
position         2686
Latitude         2686
Longitude        2686
Height_cm        2686
Weight_lbs       2686
dtype: int64

In [26]:
# Add the Players Colleges if they attended

NBAProfile = pd.merge(NBA, P_college, how='left', on=["Player","Player"])
NBAProfile['MapPt'] = pd.Series(3, index=NBAProfile.index)
# NBAProfile.count()

In [27]:
NBAProfile.set_index("Player", inplace=True)
NBAProfile.head()

,Year,Team,Team Name,Team Lat,Team Lng,Rebounds,Steals,Blocks,Assists,Points,Player Rating,State,position,Latitude,Longitude,Height_cm,Weight_lbs,College,MapPt
Player,,,,,,,,,,,,,,,,,,,
Arron Afflalo,2010.0,DEN,Denver Nuggets,39.748611,-105.007500,252.0,46.0,30.0,138.0,724.0,10.9,California,G,36.778259,-119.417931,196.0,210.0,"University of California, Los Angeles",3
LaMarcus Aldridge,2010.0,POR,Portland Trail Blazers,45.531667,-122.666667,627.0,67.0,48.0,160.0,1393.0,18.2,Texas,F-C,31.000000,-100.000000,211.0,260.0,University of Texas at Austin,3
Malik Allen,2010.0,DEN,Denver Nuggets,39.748611,-105.007500,82.0,11.0,5.0,16.0,105.0,5.9,New Jersey,F-C,39.833851,-74.871826,208.0,255.0,Villanova University,3
Ray Allen,2010.0,BOS,Boston Celtics,42.366303,-71.062228,253.0,64.0,25.0,210.0,1304.0,15.2,Florida,G,27.994402,-81.760254,203.0,205.0,University of Connecticut,3
Tony Allen,2010.0,BOS,Boston Celtics,42.366303,-71.062228,145.0,59.0,19.0,72.0,330.0,14.2,Illinois,G-F,40.000000,-89.000000,193.0,213.0,Oklahoma State University,3


In [28]:
from sqlalchemy import create_engine

In [29]:
connection_string = "postgres:postgres@localhost:5432/NBA"
engine = create_engine(f'postgresql://{connection_string}')

In [30]:
NBAProfile.to_sql(name='nbaprofile', con=engine, if_exists='replace', index=True )
engine.execute("SELECT * FROM nbaprofile").fetchall()


[('Arron Afflalo', 2010.0, 'DEN', 'Denver Nuggets', 39.748611, -105.0075, 252.0, 46.0, 30.0, 138.0, 724.0, 10.9, 'California', 'G', 36.778259000000006, -119.41793100000001, 196.0, 210.0, 'University of California, Los Angeles', 3),
 ('LaMarcus Aldridge', 2010.0, 'POR', 'Portland Trail Blazers', 45.531667, -122.66666699999999, 627.0, 67.0, 48.0, 160.0, 1393.0, 18.2, 'Texas', 'F-C', 31.0, -100.0, 211.0, 260.0, 'University of Texas at Austin', 3),
 ('Malik Allen', 2010.0, 'DEN', 'Denver Nuggets', 39.748611, -105.0075, 82.0, 11.0, 5.0, 16.0, 105.0, 5.9, 'New Jersey', 'F-C', 39.833851, -74.871826, 208.0, 255.0, 'Villanova University', 3),
 ('Ray Allen', 2010.0, 'BOS', 'Boston Celtics', 42.366303, -71.062228, 253.0, 64.0, 25.0, 210.0, 1304.0, 15.2, 'Florida', 'G', 27.994402, -81.760254, 203.0, 205.0, 'University of Connecticut', 3),
 ('Tony Allen', 2010.0, 'BOS', 'Boston Celtics', 42.366303, -71.062228, 145.0, 59.0, 19.0, 72.0, 330.0, 14.2, 'Illinois', 'G-F', 40.0, -89.0, 193.0, 213.0, 'Okla

In [31]:
# Confirm tables
engine.table_names()

['nbaprofile']

In [785]:
import gmaps
from api_keys import g_key

In [786]:
gmaps.configure(api_key=g_key)

In [831]:
birthlocations = NBAProfile[["Latitude", "Longitude"]]
Teamlocations = NBAProfile[["Team Lat", "Team Lng"]]
MP = NBAProfile["MapPt"].astype(float)

In [880]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layerB = gmaps.heatmap_layer(birthlocations, weights=(NBAProfile['Player Rating']*NBAProfile['Player Rating']), 
                                 dissipating=False, max_intensity=20000,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layerB)

# Display figure
# print(Player_Profile['State'].value_counts())
fig

Figure(layout=FigureLayout(height='420px'))

In [793]:
# Plot Heatmap
fig2 = gmaps.figure()

# Create heat layer
heat_layerT = gmaps.heatmap_layer(Teamlocations, weights=MP, 
                                 dissipating=False, max_intensity=250,
                                 point_radius=1)
# Add layer
fig2.add_layer(heat_layerT)

# Display figure
# print(Player_Profile['State'].value_counts())
fig2

Figure(layout=FigureLayout(height='420px'))